In [8]:
# This notebook defines some functions that are used to calculate AUROC and AUPR for each scored screen.
# Two benchmarks are also loaded and processed here. 

In [2]:
%matplotlib inline
import pandas as pd
import scipy.stats as stats
import scipy.cluster.hierarchy as clust
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn import metrics
from itertools import combinations
from scipy.stats import ttest_rel
from statsmodels.stats.multitest import multipletests


In [10]:
# Pre-define list of the scoring methods
scores = ['zdLFC', 'Orthrus',  'Gemini(Strong)','Gemini(Sens)', 'Parrish']

In [3]:
# This function takes in each dataframe of gene pairs score and alphabetically sorts the gene pairs, such that if the A1<A2,
# the gene pair is A1_A2, else its A2_A1
def reindex_alphbetically(df):
    df['gene_pair'] = df.index
    
    for index, row in df.iterrows():
        a, b = index.split('_')
        if a < b:
            df.at[index, 'gene_pair'] = f'{a}_{b}'
        else:
            df.at[index, 'gene_pair'] = f'{b}_{a}'
    df.set_index('gene_pair', inplace = True)     
    return(df)

In [12]:
# Function to calculate Jaccard similarity
def jaccard_similarity(df, percentile):
        
    def keep_top_X_percent(series, percentile):
        threshold = series.quantile(percentile)  # 95th percentile
        #print(threshold)
        return series.where(series >= threshold, np.nan)
    
    def CalculateSimilarity(set1,set2):
        set1 = set(set1)
        set2 = set(set2)
        intersection = len(set1.intersection(set2))
        union = len(set1.union(set2))
        return intersection / union if union != 0 else np.nan
    # Calculate pairwise Jaccard similarities

    
    df = df.apply(lambda col: keep_top_X_percent(col, percentile), axis=0)
    #display(df)
    # Extract non-NA indices for each column
    non_na_indices = {col: df[df[col].notna()].index.tolist() for col in df.columns}
    
    # Convert dictionary to DataFrame for better viewing
    df_indices = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in non_na_indices.items()]))
    #display(df_indices)
    # Create a DataFrame to store Jaccard similarities
    columns = df_indices.columns
    
    jaccard_matrix = pd.DataFrame(np.nan, index=columns, columns=columns)

    for col1, col2 in combinations(columns, 2):
        
        similarity = CalculateSimilarity(df_indices[col1], df_indices[col2])
        jaccard_matrix.at[col1, col2] = similarity
        jaccard_matrix.at[col2, col1] = similarity
    





    # Fill diagonal with 1 since Jaccard similarity with itself is always 1
    np.fill_diagonal(jaccard_matrix.values, 1)
    return jaccard_matrix
    




In [5]:
def ROC_Curve(scores, labels, ground_truth, title = "", plot = True):
    results = []
    
    if plot:
        plt.figure(0).clf()
        plt.ylabel('True Positive Rate')
        plt.xlabel('False Positive Rate')
        plt.plot([0, 1], [0, 1], color='black', lw=2, linestyle='--', label = "random")
    
    
    for score, label in zip(scores, labels):   
        data = { 'ground_truth': ground_truth,
                'score': score}
        
        
        df = pd.DataFrame(data)

        # Remove rows with NaN values in either column
        
        df_clean = df.dropna()
        
        if (df_clean.shape[0] == 0):
            print("Error - All NA's here")
            fpr = np.nan
            tpr = np.nan
            auc = np.nan
        else:
            fpr, tpr, _ = metrics.roc_curve(df_clean['ground_truth'],  df_clean['score'])
            auc = metrics.auc(fpr, tpr)
            
            #create ROC curve
            if plot:
                plt.plot(fpr,tpr, label=label + ", auc="+str(auc.round(2)) + " n=" + str(df_clean.shape[0]))
            
        results.append((auc, label,df_clean.shape[0],df_clean['ground_truth'].sum()))
    if plot:   
        plt.legend(loc='lower center', bbox_to_anchor=(0.5, -0.5), shadow=True, ncol=2)    
        plt.title(title)    
        plt.show()


    return results

def PR_Curve(scores, labels, ground_truth, title = "", plot = True):
    results = []
    baseline = sum(ground_truth)/len(ground_truth)

    if plot:
        plt.figure(0).clf()
        plt.ylabel('Precision')
        plt.xlabel('Recall')
        plt.plot([0, 1], [baseline, baseline], color='black', lw=2, linestyle='--', label = "baseline")

    for score, label in zip(scores, labels):    
        data = { 'ground_truth': ground_truth,
                'score': score}
        
        
        df = pd.DataFrame(data)

        # Remove rows with NaN values in either column
        df_clean = df.dropna()
        

        if (df_clean.shape[0] > 0):
            lr_precision, lr_recall, _ = metrics.precision_recall_curve(df_clean['ground_truth'],  df_clean['score'])
            lr_auc = metrics.auc(lr_recall, lr_precision)
            
                
            if plot:   
                plt.plot(lr_recall,lr_precision, label=label + ", auc="+ str(lr_auc.round(2)) + " n=" + str(df_clean.shape[0]))

        else:
            print("Error - All NA's here")
            lr_precision = np.nan
            lr_recall = np.nan
            lr_auc = np.nan


        results.append((lr_auc, label,df_clean.shape[0],df_clean['ground_truth'].sum()))

    if plot:
        plt.legend(loc='lower center', bbox_to_anchor=(0.5, -0.5), shadow=True, ncol=2)
        plt.title(title)
        plt.show()
    return results

In [ ]:
# For each study/cell line, this function calls the ROC_Curve and PR_curve functions
def Cell_Line_Analysis(df,cell_line_name, study_name, labels, ground_truth, plot = True):
    
    ground_truth.index.name = "sorted_gene_pair"

    ground_truth = ground_truth.reset_index()

    # Group by the original index and take max of ground_truth (since 1 > 0)
    ground_truth = ground_truth.groupby('sorted_gene_pair', as_index=False).agg({'ground_truth': 'max'})

    # Set index back
    ground_truth = ground_truth.set_index('sorted_gene_pair')



    df = df.join(ground_truth, how='left')
    df = df.dropna(subset=['ground_truth'])

    ground_truth = df["ground_truth"]
       
    title = study_name + ": " + cell_line_name
    
    
    scores =  [df[column].tolist() for column in df.columns]
    
    roc_auc = ROC_Curve(scores, labels, ground_truth, title, plot =plot) 
    pr_auc = PR_Curve(scores, labels ,ground_truth, title, plot =plot)

    return roc_auc, pr_auc, cell_line_name, study_name


In [15]:
# Takes in data returned by the Cell_Line_Analysis function and organises it in form of a df. 
def ConvertResultsToDF(list_of_results):
    dfs_roc = []
    dfs_pr = []

    for result_list in list_of_results:
        
        roc = result_list[0]
        pr = result_list[1]
        cell_line = result_list[2]
        study_name = result_list[3]
        
        #ROC
        
        roc = pd.DataFrame(roc, columns=['value', 'Score', 'Common samples', 'Positive Samples'])
       
        roc['Cell line'] = cell_line
        roc['Study name'] = study_name

        dfs_roc.append(roc)
        
        #PR
        
        pr = pd.DataFrame(pr, columns=['value', 'Score', 'Common samples', 'Positive Samples'])

        pr['Cell line'] = cell_line
        pr['Study name'] = study_name
        dfs_pr.append(pr)
        
        
         
        
    big_df_roc = pd.concat(dfs_roc, ignore_index=True)
    big_df_pr = pd.concat(dfs_pr, ignore_index=True)

    return big_df_roc, big_df_pr



In [16]:
# Load Ground truth
# Data from 10.1016/j.cels.2021.08.006 
barbaras = pd.read_csv('../InputData/Benchmarks/deKegel_output.csv', sep=',', index_col = 2)
barbaras.head()
ground_truth_depmap_hit = barbaras[['depmap_hit']]
ground_truth_depmap_hit = ground_truth_depmap_hit.replace({True: 1, False: 0})
ground_truth_depmap_hit = ground_truth_depmap_hit.rename(columns={'depmap_hit': 'ground_truth'})
ground_truth_depmap_hit = ground_truth_depmap_hit.dropna()
ground_truth_depmap_hit.to_csv('../InputData/Benchmarks/processed/ground_truth_depmap_hit_processed.csv', index=True)

C:\Users\hamda\AppData\Local\Temp\ipykernel_42040\1198573690.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  ground_truth_depmap_hit = ground_truth_depmap_hit.replace({True: 1, False: 0})


In [4]:
# Ground Truth 2
# Data is downloaded from : DOI: 10.1016/j.celrep.2022.110636

Koferle = pd.read_excel("../InputData/Benchmarks/Koferle.xlsx", sheet_name='PaCT')
Koferle[['gene1', 'gene2']] = Koferle['Pair'].str.split('_', expand=True)

# Use apply along axis=1 to reorder the gene names alphabetically and reconstruct the 'Pair'
Koferle['Pair'] = Koferle.apply(lambda x: '_'.join(sorted([x['gene1'], x['gene2']])), axis=1)
Koferle.rename(columns = {'Pair':'sorted_gene_pair'}, inplace = True)

#display(Koferle[Koferle.duplicated('sorted_gene_pair', keep=False)])
Koferle = Koferle[['sorted_gene_pair', 'Sub-Dataset', 'Correlation Spearman', 'Screen Type', 'Hit?']]
Koferle.set_index('sorted_gene_pair', inplace=True)
#Koferle

Koferle = Koferle.copy()
Koferle = Koferle[Koferle['Screen Type'] == 'AVANA']
Koferle = Koferle[Koferle['Sub-Dataset'] == 'expr_paralog']#
Koferle['ground_truth'] = np.nan
Koferle.loc[(Koferle['Correlation Spearman'] > 0) & (Koferle['Hit?'] == 'yes'), ['ground_truth']] = 1
Koferle.loc[(Koferle['Correlation Spearman'] < 0) & (Koferle['Hit?'] == 'no'), ['ground_truth']] = 0
Koferle = Koferle.dropna(subset=['ground_truth'])

Koferle = Koferle[['ground_truth']]
display(Koferle[Koferle.index.duplicated(keep=False)])
Koferle.to_csv('../InputData/Benchmarks/processed/ground_truth_koferle_processed.csv', index=True)

,ground_truth
sorted_gene_pair,
A1BG_GP6,0.0
A1BG_IGSF1,0.0
A1BG_KIR2DL3,0.0
A1BG_LILRB3,0.0
A1BG_LILRB5,0.0
...,...
ZSWIM4_ZSWIM5,0.0
ZER1_ZYG11A,0.0
ZER1_ZYG11B,0.0


In [ ]:
Koferle = pd.read_csv('../InputData/Benchmarks/processed/ground_truth_koferle_processed.csv', index_col=0)

In [ ]:
bacon = pd.read_excel("C:\\Users\\hamda\\OneDrive\\Documents\\GitHub\\PostDoc\\Conway\\Research\\GIScoring\\Bacon\\data\\ev2.xlsx")
bacon.rename(columns = {'buffered_gene': 'gene1', 'buffering_gene': 'gene2'}, inplace = True)
bacon['sorted_gene_pair'] = bacon.apply(lambda x: '_'.join(sorted([x['gene1'], x['gene2']])), axis=1)
bacon = bacon[['sorted_gene_pair']]
bacon['ground_truth'] = 1
bacon = bacon.drop_duplicates()
bacon.set_index('sorted_gene_pair', inplace=True)

bacon.to_csv('../InputData/Benchmarks/processed/ground_truth_bacon_processed.csv', index=True)

In [1]:
def Cell_Line_Analysis_bacon(df,cell_line_name, study_name, labels, ground_truth, plot = True):
    df = df.dropna()
    df = df.join(ground_truth, how='left')
    

    df['ground_truth'] = df['ground_truth'].fillna(0) # everything else is considered a negative
    #display(df)
    ground_truth = df["ground_truth"]

    title = study_name + ": " + cell_line_name

    scores =  [df[column].tolist() for column in df.columns]

    roc_auc = ROC_Curve(scores, labels, ground_truth, title, plot =plot)
    pr_auc = PR_Curve(scores, labels ,ground_truth, title, plot =plot)

    return roc_auc, pr_auc, cell_line_name, study_name

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm
def stratified_sample(df, label_col='ground_truths', frac=0.75, random_state=None):
    sampled_parts = [
        group.sample(frac=frac, random_state=random_state)
        for _, group in df.groupby(label_col)
    ]
    return pd.concat(sampled_parts).sample(frac=1, random_state=random_state).reset_index(drop=True)


def Resampling_analysis(df,cell_line_name, study_name, labels, ground_truth, n_repeats=500, frac = 0.75, alpha = 0.05):
    

    results = pd.DataFrame()

    df = df.join(ground_truth, how='left')
    df = df.dropna(subset=['ground_truth'])



    for i in range(n_repeats):
        df_sampled = stratified_sample(df, label_col='ground_truth', frac=frac, random_state=i)

        ground_truth_sampled = df_sampled["ground_truth"]
        scores =  [df_sampled[column].tolist() for column in df_sampled.columns]

        roc_auc = ROC_Curve(scores, labels, ground_truth_sampled, "", plot =False) 
        pr_auc = PR_Curve(scores, labels ,ground_truth_sampled, "", plot =False) 
        # Flatten into a single list
        roc_auc = pd.DataFrame(roc_auc, columns=['AUROC', 'method','total','ground_truth'])
        pr_auc = pd.DataFrame(pr_auc, columns=['AUPR', 'method','total','ground_truth'])

        merged_df = pd.merge(roc_auc, pr_auc, on=['method','total','ground_truth'], how='inner')
        #print(merged_df)

        #flat_result = [roc_auc[0] ,roc_auc[1] , pr_auc[0] , pr_auc[1] ,cell_line_name, study_name]
        # Collect results
        merged_df['replicate'] = i
        results = pd.concat([results, merged_df], ignore_index=True)
    #display(results)
    # Best method by mean AUROC
    best_auroc_method = results.groupby("method")["AUROC"].mean().idxmax()
    # Best method by mean AUPR
    best_aupr_method = results.groupby("method")["AUPR"].mean().idxmax()


    print(f"Best AUROC Method: {best_auroc_method}")
    print(f"Best AUPR Method: {best_aupr_method}")
 

    aupr_test = compare_methods_ttest(results, "AUPR", best_aupr_method,alpha=alpha)
    auroc_test = compare_methods_ttest(results, "AUROC", best_auroc_method,alpha=alpha)

    df = results.copy()
    for metric in ['AUPR', 'AUROC']:
        plt.figure()
        for score in df['method'].unique():
            data = df[df['method'] == score][metric].dropna()
            if len(data) < 2:
                continue  # Skip if not enough data

            sns.kdeplot(data, label=f'Score: {score}', fill=True, alpha=0.3)

        plt.title(f'KDE of {metric.upper()} by Method')
        plt.xlabel(metric.upper())
        plt.ylabel('Density')
        plt.legend()
        plt.grid(True)
        plt.show()

    
    return pd.DataFrame(results), aupr_test, auroc_test




def compare_methods_ttest(df, metric, best_method, alpha):
    results = []

    # Pivot table: rows = replicates, columns = methods
    pivot = df.pivot(index="replicate", columns="method", values=metric)

    for method in pivot.columns:
        if method == best_method:
            continue

        x = pivot[best_method]
        y = pivot[method]

        # Paired t-test
        stat, p = ttest_rel(x, y, nan_policy='omit')
        results.append((method, p))

    # Adjust p-values for multiple comparisons
    methods, pvals = zip(*results)
    fdr = multipletests(pvals, method='fdr_bh')[1]
    bonferroni = multipletests(pvals, method='bonferroni')[1]
    significant_bh = [adj < alpha for adj in fdr]
    significant_bonferroni = [adj < alpha for adj in bonferroni]

    return pd.DataFrame({
        "method": methods,
        "p_value": pvals,
        "fdr": fdr,
        "bonferroni": bonferroni,
        "significant_bh": significant_bh,
        'signicant_bonferroni': significant_bonferroni

    }).sort_values("fdr")